In [1]:
import numpy as np
from scipy.sparse import lil_matrix

In [2]:
def shapeFn1d(i, x, x1, x2, p):
    """
    TASK   - Compute value of ith shape function on element [x1,x2]

    INPUTS - i     : i(th) shape function and its derivative on [x1,x2]
             x     : input value for the shape function psi_i(x)
             x1,x2 : endpoints of the element, where x1<x2
             p     : order of the shape function
    OUTPUT - y     : value
    """
    if x < x1 or x > x2:
        return 0  # When x is not in [x1,x2], value is 0

    # Linear Shape Functions
    if p == 1:
        if i == 1:
            return (x2 - x) / (x2 - x1)
        elif i == 2:
            return (x - x1) / (x2 - x1)
        else:
            raise ValueError("Invalid shape function index for p=1.")

    # Quadratic Shape Functions
    elif p == 2:
        mid = (x1 + x2) / 2  # Middle point of the element
        if i == 1:
            return (x - mid) * (x - x2) / ((x1 - mid) * (x1 - x2))
        elif i == 2:
            return (x - x1) * (x - x2) / ((mid - x1) * (mid - x2))
        elif i == 3:
            return (x - x1) * (x - mid) / ((x2 - x1) * (x2 - mid))
        else:
            raise ValueError("Invalid shape function index for p=2.")
    else:
        raise ValueError("Only linear (p=1) and quadratic (p=2) shape functions are considered.")

def shapeFnDer1d(i, x, x1, x2, p):
    """
    TASK   - Compute derivative of ith shape function on element [x1, x2]

    INPUTS - i     : i(th) shape function and its derivative on [x1,x2]
             x     : input value for the shape function psi_i(x)
             x1,x2 : endpoints of the element, where x1<x2
             p     : order of the shape function
    OUTPUT - y     : value
    """
    if x < x1 or x > x2:
        return 0  # When x is not in [x1,x2], value is 0

    # Derivatives of Linear Shape Functions
    if p == 1:
        if i == 1:
            return -1 / (x2 - x1)
        elif i == 2:
            return 1 / (x2 - x1)
        else:
            raise ValueError("Invalid shape function index for p=1.")

    # Derivatives of Quadratic Shape Functions
    elif p == 2:
        mid = (x1 + x2) / 2  # Midpoint of the element
        if i == 1:
            return ((x - x2) + (x - mid)) / ((x1 - mid) * (x1 - x2))
        elif i == 2:
            return ((x - x2) + (x - x1)) / ((mid - x1) * (mid - x2))
        elif i == 3:
            return ((x - mid) + (x - x1)) / ((x2 - x1) * (x2 - mid))
        else:
            raise ValueError("Invalid shape function index for p=2.")

    else:
        raise ValueError("Only linear (p=1) and quadratic (p=2) shape functions are considered.")

In [3]:
 def gaussQuadStd1d(g, noOfIntegPt):
    """
    TASK   - Evaluate definite integral of G(ξ) from -1 to 1 using Gaussian quadrature

    INPUTS - g           : function representing the integrand
             noOfIntegPt : number of integration points
    OUTPUT - y           : value of integral
    """
    if noOfIntegPt == 2:
        # Coefficients and points for 2-point Gaussian quadrature
        weights = [1.0, 1.0]
        points = [-1/(3 ** 0.5) , 1/(3 ** 0.5)]
    elif noOfIntegPt == 3:
        # Coefficients and points for 3-point Gaussian quadrature
        weights = [5/9, 8/9, 5/9]
        points = [-((3 / 5) ** 0.5), 0.0, ((3 / 5) ** 0.5)]
    else:
        raise ValueError("Only 2 or 3 integration points are supported.")

    # Compute integral using quadrature formula
    integral = 0.0
    for i in range(len(points)):
        integral += weights[i] * g(points[i])

    return integral

In [4]:
def gaussQuad1d(fn, lowerLimit, upperLimit, noOfIntegPt):
    """
    TASK   - Evaluate definite integral of fn(x) from lowerLimit to upperLimit using Gaussian quadrature

    INPUTS - fn          : function representing the integrand
             lowerLimit  : lower integration limit (a)
             upperLimit  : upper integration limit (b)
             noOfIntegPt : number of integration points
    OUTPUT - y           : value of integral
    """
    # Create transformation from [a, b] to [-1, 1]
    def transform(xi):
        return 0.5 * (upperLimit - lowerLimit) * xi + 0.5 * (upperLimit + lowerLimit)

    # Define transformed integrand
    def transformedfn(xi):
        x = transform(xi)  # Change of variables x to ξ
        return ( (upperLimit - lowerLimit) / 2 ) * fn(x)

    # Use gaussQuadStd1d to compute integral on [-1, 1]
    return gaussQuadStd1d(transformedfn, noOfIntegPt)

In [5]:
def meij(e, i, j, xh, shapeFn, noOfIntegPt):
    """
    TASK   - Evaluate (i, j)-th entry of the element mass matrix M^e

    INPUTS - e           : element index
             i,j         : indices for the mass matrix entry
             xh          : array of element nodes [x_1, x_2, ..., x_n+1]
             shapeFn     : degree of shape functions (1 = linear, 2 = quadratic)
             noOfIntegPt : number of integration points for Gaussian quadrature
    OUTPUT - y           : value
    """
    x1 = xh[e - 1]  # Left boundary of the e-th element
    x2 = xh[e]      # Right boundary of the e-th element

    def integrand(x):
        # Product of shape functions psi_i and psi_j
        return shapeFn1d(i, x, x1, x2, shapeFn) * shapeFn1d(j, x, x1, x2, shapeFn)

    # Perform integration using Gaussian quadrature
    y = gaussQuad1d(integrand, x1, x2, noOfIntegPt)
    return y


def keij(a, c, e, i, j, xh, shapeFn, noOfIntegPt):
    """
    TASK   - Evaluate (i, j)-th entry of element stiffness matrix K^e

    INPUTS - a           : function 
             c           : function 
             e           : element index
             i,j         : indices for the stiffness matrix entry
             xh          : array of element nodes [x_1, x_2, ..., x_n+1]
             shapeFn     : degree of shape functions (1 = linear, 2 = quadratic)
             noOfIntegPt : number of integration points for Gaussian quadrature
    OUTPUT - y           : value
    """
    x1 = xh[e - 1]  # Left boundary of the e-th element
    x2 = xh[e]      # Right boundary of the e-th element

    def integrand(x):
        # Retrieve shape functions and derivatives
        psi_i = shapeFn1d(i, x, x1, x2, shapeFn)
        psi_j = shapeFn1d(j, x, x1, x2, shapeFn)
        psi_i_Der = shapeFnDer1d(i, x, x1, x2, shapeFn)
        psi_j_Der = shapeFnDer1d(j, x, x1, x2, shapeFn)

        # Contribution from derivative-dependent and non-derivative terms
        return a(x) * psi_i_Der * psi_j_Der + c(x) * psi_i * psi_j

    # Perform integration using Gaussian quadrature
    y = gaussQuad1d(integrand, x1, x2, noOfIntegPt)
    
    return y


def fei(a, c, f, p0, e, i, xh, shapeFn, noOfIntegPt):
    """
    TASK   - Evaluate i-th entry of element load vector F^e

    INPUTS - a           : function 
             c           : function 
             f           : load function in boundary value problem
             p0          : dirichlet boundary condition for solution at x=0
             e           : element index
             i           : index for the load vector entry
             xh          : array of element nodes [x_1, x_2, ..., x_n+1]
             shapeFn     : degree of shape functions (1 = linear, 2 = quadratic)
             noOfIntegPt : number of integration points for Gaussian quadrature
    OUTPUT - y           : value
    """
    x1 = xh[e - 1]  # Left boundary of the e-th element
    x2 = xh[e]      # Right boundary of the e-th element

    def integrand(x):
        # Retrieve shape functions and derivatives
        psi_i = shapeFn1d(i, x, x1, x2, shapeFn)
        psi_i_Der = shapeFnDer1d(i, x, x1, x2, shapeFn)
        
        # Product of load function and shape function psi_i
        return ( f(x)*psi_i 
                - (a(x)*p0*shapeFnDer1d(1, x, xh[0], xh[1], shapeFn)*psi_i_Der) 
                - (c(x)*p0*shapeFn1d(1, x, xh[0], xh[1], shapeFn)*psi_i)
               )

    # Perform numerical integration using Gaussian quadrature
    y = gaussQuad1d(integrand, x1, x2, noOfIntegPt)

    return y

In [6]:
def L2norm1d(f, a, b, noOfEle):
    """
    TASK - Compute L2 norm of a one-variable L2 function using Gaussian quadrature

    INPUTS - f        : function 
             a, b     : endpoints of the domain [a, b]
             noOfEle  : number of subintervals to divide [a, b]
    OUTPUT - norm     : computed L2 norm (scalar)
    """
    # Subinterval size
    h = (b - a) / noOfEle

    # Initialize norm
    norm = 0.0

    for e in range(noOfEle):
        # Define element endpoints
        x_start = a + e * h
        x_end = x_start + h

        # Define integrand for current subinterval (f(x)^2)
        def integrand(x):
            return f(x) ** 2

        # Use gaussQuad1d to integrate over current subinterval
        integral = gaussQuad1d(integrand, x_start, x_end, 3)  # 3 integration points

        norm += integral

    # Compute square root of integral
    return np.sqrt(norm)

In [7]:
def massM(xh, shapeFn, noOfIntegPt):
    """
    Construct global consistent mass matrix M without extra zero row/column
    
    Parameters:
    xh : array of element nodes [x_1, x_2, ..., x_{n+1}]
    shapeFn : degree of shape functions (1 = linear, 2 = quadratic)
    noOfIntegPt : number of integration points for Gaussian quadrature
    
    Returns:
    M : sparse matrix representing global mass matrix without extra zeros
    """
    
    n = len(xh) - 1  # number of elements
    
    # Determine matrix size based on shape functions
    if shapeFn == 1:
        size = n  # For linear, we have n+1 nodes but first node is Dirichlet
    elif shapeFn == 2:
        size = 2 * n  # For quadratic, we have 2n+1 nodes but first node is Dirichlet
    else:
        raise ValueError("shapeFn must be 1 (linear) or 2 (quadratic)")
    
    M = lil_matrix((size, size))
    
    for e in range(1, n + 1):
        if shapeFn == 1:
            # Linear shape functions (2 nodes per element)
            if e == 1:
                # Only right node contributes (left node has Dirichlet condition)
                M[0, 0] += meij(e, 2, 2, xh, shapeFn, noOfIntegPt)
            else:
                i_global = e - 1
                M[i_global-1, i_global-1] += meij(e, 1, 1, xh, shapeFn, noOfIntegPt)
                M[i_global-1, i_global] += meij(e, 1, 2, xh, shapeFn, noOfIntegPt)
                M[i_global, i_global-1] += meij(e, 2, 1, xh, shapeFn, noOfIntegPt)
                M[i_global, i_global] += meij(e, 2, 2, xh, shapeFn, noOfIntegPt)

        elif shapeFn == 2:
            # Quadratic shape functions (3 nodes per element)
            if e == 1:
                # First element: skip the first DOF (Dirichlet condition)
                M[0, 0] += meij(e, 2, 2, xh, shapeFn, noOfIntegPt)
                M[0, 1] += meij(e, 2, 3, xh, shapeFn, noOfIntegPt)
                M[1, 0] += meij(e, 3, 2, xh, shapeFn, noOfIntegPt)
                M[1, 1] += meij(e, 3, 3, xh, shapeFn, noOfIntegPt)
            else:
                i_global = 2*(e-1) - 1
                
                # Row 1
                M[i_global, i_global] += meij(e, 1, 1, xh, shapeFn, noOfIntegPt)
                M[i_global, i_global+1] += meij(e, 1, 2, xh, shapeFn, noOfIntegPt)
                M[i_global, i_global+2] += meij(e, 1, 3, xh, shapeFn, noOfIntegPt)
                
                # Row 2
                M[i_global+1, i_global] += meij(e, 2, 1, xh, shapeFn, noOfIntegPt)
                M[i_global+1, i_global+1] += meij(e, 2, 2, xh, shapeFn, noOfIntegPt)
                M[i_global+1, i_global+2] += meij(e, 2, 3, xh, shapeFn, noOfIntegPt)
                
                # Row 3
                M[i_global+2, i_global] += meij(e, 3, 1, xh, shapeFn, noOfIntegPt)
                M[i_global+2, i_global+1] += meij(e, 3, 2, xh, shapeFn, noOfIntegPt)
                M[i_global+2, i_global+2] += meij(e, 3, 3, xh, shapeFn, noOfIntegPt)
    
    return M

In [8]:
def stiffK(a, c, xh, shapeFn, noOfIntegPt):
    """
    Construct global stiffness matrix K without extra zero row/column
    
    Parameters:
    a, c : coefficient functions
    xh : array of element nodes [x_1, x_2, ..., x_{n+1}]
    shapeFn : degree of shape functions (1 = linear, 2 = quadratic)
    noOfIntegPt : number of integration points for Gaussian quadrature
    
    Returns:
    K : sparse matrix representing global stiffness matrix without extra zeros
    """
    
    n = len(xh) - 1  # number of elements
    
    # Determine matrix size based on shape functions
    if shapeFn == 1:
        size = n  # For linear, we have n+1 nodes but first node is Dirichlet
    elif shapeFn == 2:
        size = 2 * n  # For quadratic, we have 2n+1 nodes but first node is Dirichlet
    else:
        raise ValueError("shapeFn must be 1 (linear) or 2 (quadratic)")
    
    K = lil_matrix((size, size))
    
    for e in range(1, n + 1):
        if shapeFn == 1:
            # Linear shape functions
            if e == 1:
                # Only right node contributes (left node has Dirichlet condition)
                K[0, 0] += keij(a, c, e, 2, 2, xh, shapeFn, noOfIntegPt)
            else:
                i_global = e - 1
                K[i_global-1, i_global-1] += keij(a, c, e, 1, 1, xh, shapeFn, noOfIntegPt)
                K[i_global-1, i_global] += keij(a, c, e, 1, 2, xh, shapeFn, noOfIntegPt)
                K[i_global, i_global-1] += keij(a, c, e, 2, 1, xh, shapeFn, noOfIntegPt)
                K[i_global, i_global] += keij(a, c, e, 2, 2, xh, shapeFn, noOfIntegPt)

        elif shapeFn == 2:
            # Quadratic shape functions
            if e == 1:
                # First element: skip the first DOF (Dirichlet condition)
                K[0, 0] += keij(a, c, e, 2, 2, xh, shapeFn, noOfIntegPt)
                K[0, 1] += keij(a, c, e, 2, 3, xh, shapeFn, noOfIntegPt)
                K[1, 0] += keij(a, c, e, 3, 2, xh, shapeFn, noOfIntegPt)
                K[1, 1] += keij(a, c, e, 3, 3, xh, shapeFn, noOfIntegPt)
            else:
                i_global = 2*(e-1) - 1
                
                # Row 1
                K[i_global, i_global] += keij(a, c, e, 1, 1, xh, shapeFn, noOfIntegPt)
                K[i_global, i_global+1] += keij(a, c, e, 1, 2, xh, shapeFn, noOfIntegPt)
                K[i_global, i_global+2] += keij(a, c, e, 1, 3, xh, shapeFn, noOfIntegPt)
                
                # Row 2
                K[i_global+1, i_global] += keij(a, c, e, 2, 1, xh, shapeFn, noOfIntegPt)
                K[i_global+1, i_global+1] += keij(a, c, e, 2, 2, xh, shapeFn, noOfIntegPt)
                K[i_global+1, i_global+2] += keij(a, c, e, 2, 3, xh, shapeFn, noOfIntegPt)
                
                # Row 3
                K[i_global+2, i_global] += keij(a, c, e, 3, 1, xh, shapeFn, noOfIntegPt)
                K[i_global+2, i_global+1] += keij(a, c, e, 3, 2, xh, shapeFn, noOfIntegPt)
                K[i_global+2, i_global+2] += keij(a, c, e, 3, 3, xh, shapeFn, noOfIntegPt)
    
    return K

In [9]:
def loadF(a, c, f, p0, QL, xh, shapeFn, noOfIntegPt):
    """
    Construct global load vector F without extra zero entry
    
    Parameters:
    a, c, f : coefficient functions
    p0 : Dirichlet BC at x=0
    QL : Neumann BC at x=L (a * du/dn)
    xh : array of element nodes [x_1, x_2, ..., x_{n+1}]
    shapeFn : degree of shape functions (1 = linear, 2 = quadratic)
    noOfIntegPt : number of integration points for Gaussian quadrature
    
    Returns:
    F : 1D numpy array representing global load vector without extra zeros
    """
    
    n = len(xh) - 1  # number of elements
    
    # Determine vector size based on shape functions
    if shapeFn == 1:
        size = n  # For linear, we have n+1 nodes but first node is Dirichlet
    elif shapeFn == 2:
        size = 2 * n  # For quadratic, we have 2n+1 nodes but first node is Dirichlet
    else:
        raise ValueError("shapeFn must be 1 (linear) or 2 (quadratic)")
    
    F = np.zeros(size)
    
    for e in range(1, n + 1):
        if shapeFn == 1:
            # Linear shape functions
            if e == 1:
                # Only right node contributes (left node has Dirichlet condition)
                F[0] += fei(a, c, f, p0, e, 2, xh, shapeFn, noOfIntegPt)
            else:
                i_global = e - 1
                F[i_global-1] += fei(a, c, f, p0, e, 1, xh, shapeFn, noOfIntegPt)
                F[i_global] += fei(a, c, f, p0, e, 2, xh, shapeFn, noOfIntegPt)

        elif shapeFn == 2:
            # Quadratic shape functions
            if e == 1:
                # First element: skip the first DOF (Dirichlet condition)
                F[0] += fei(a, c, f, p0, e, 2, xh, shapeFn, noOfIntegPt)
                F[1] += fei(a, c, f, p0, e, 3, xh, shapeFn, noOfIntegPt)
            else:
                i_global = 2*(e-1) - 1
                F[i_global] += fei(a, c, f, p0, e, 1, xh, shapeFn, noOfIntegPt)
                F[i_global+1] += fei(a, c, f, p0, e, 2, xh, shapeFn, noOfIntegPt)
                F[i_global+2] += fei(a, c, f, p0, e, 3, xh, shapeFn, noOfIntegPt)
    
    # Add Neumann boundary condition at x=L
    if shapeFn == 1:
        F[-1] += QL  # Last node for linear
    elif shapeFn == 2:
        F[-1] += QL  # Last node for quadratic
    
    return F

In [10]:
a = lambda x: 3*x + x**2
c = lambda x: x
f = lambda x: np.sin(x)
p0 = 2
xh = np.linspace(0.,2.,6)
print(f'meij(4,1,2,xh,1,3) = { meij(4,1,2,xh,1,3) }')
print(f'keij(a,c,4,1,2,xh,1,3) = {keij(a,c,4,1,2,xh,1,3) }')
print(f'fei(a,c,f,p0,1,2,xh,1,3) = {fei(a,c,f,p0,1,2,xh,1,3)}')

meij(4,1,2,xh,1,3) = 0.06666666666666668
keij(a,c,4,1,2,xh,1,3) = -15.340000000000003
fei(a,c,f,p0,1,2,xh,1,3) = 3.2924848499213506


In [11]:
print(f'massM(xh, 1, 2) = ')
print(f'{massM(xh, 1, 2) }')
M = massM(xh, 1, 2).toarray()
print(M)

massM(xh, 1, 2) = 
  (0, 0)	0.2666666666666667
  (0, 1)	0.06666666666666665
  (1, 0)	0.06666666666666665
  (1, 1)	0.26666666666666694
  (1, 2)	0.06666666666666668
  (2, 1)	0.06666666666666668
  (2, 2)	0.2666666666666666
  (2, 3)	0.06666666666666662
  (3, 2)	0.06666666666666662
  (3, 3)	0.2666666666666667
  (3, 4)	0.06666666666666662
  (4, 3)	0.06666666666666662
  (4, 4)	0.13333333333333336
[[0.26666667 0.06666667 0.         0.         0.        ]
 [0.06666667 0.26666667 0.06666667 0.         0.        ]
 [0.         0.06666667 0.26666667 0.06666667 0.        ]
 [0.         0.         0.06666667 0.26666667 0.06666667]
 [0.         0.         0.         0.06666667 0.13333333]]


In [12]:
print(f'stiffK(a, c, xh, 1, 2) = ')
print(f'{stiffK(a, c, xh, 1, 2)}')
K = stiffK(a, c, xh, 1, 2).toarray()
print(K)

stiffK(a, c, xh, 1, 2) = 
  (0, 0)	7.173333333333336
  (0, 1)	-5.393333333333334
  (1, 0)	-5.393333333333334
  (1, 1)	15.679999999999998
  (1, 2)	-9.966666666666661
  (2, 1)	-9.966666666666661
  (2, 2)	25.786666666666665
  (2, 3)	-15.340000000000003
  (3, 2)	-15.340000000000003
  (3, 3)	37.49333333333334
  (3, 4)	-21.51333333333334
  (4, 3)	-21.51333333333334
  (4, 4)	21.88666666666667
[[  7.17333333  -5.39333333   0.           0.           0.        ]
 [ -5.39333333  15.68        -9.96666667   0.           0.        ]
 [  0.          -9.96666667  25.78666667 -15.34         0.        ]
 [  0.           0.         -15.34        37.49333333 -21.51333333]
 [  0.           0.           0.         -21.51333333  21.88666667]]


In [13]:
print(f'loadF(a,c,f,p0,-3,xh,1, 2) = {loadF(a, c, f, p0, -3, xh, 1, 2)}')
F = loadF(a, c, f, p0, -3, xh, 1, 2)
print(F)

loadF(a,c,f,p0,-3,xh,1, 2) = [ 3.39370424  0.28314196  0.36787779  0.39453381 -2.80955013]
[ 3.39370424  0.28314196  0.36787779  0.39453381 -2.80955013]


In [14]:
xh = np.linspace(0.,1.5,4)
print(f'massM(xh, 2, 2) = ')
print(f'{massM(xh, 2, 2) }')
M = massM(xh, 2, 2).toarray()
print(M)

massM(xh, 2, 2) = 
  (0, 0)	0.2222222222222222
  (0, 1)	0.055555555555555566
  (1, 0)	0.055555555555555566
  (1, 1)	0.11111111111111108
  (1, 2)	0.05555555555555555
  (1, 3)	-0.027777777777777776
  (2, 1)	0.05555555555555555
  (2, 2)	0.22222222222222227
  (2, 3)	0.05555555555555555
  (3, 1)	-0.027777777777777776
  (3, 2)	0.05555555555555555
  (3, 3)	0.11111111111111119
  (3, 4)	0.055555555555555594
  (3, 5)	-0.027777777777777797
  (4, 3)	0.055555555555555594
  (4, 4)	0.2222222222222219
  (4, 5)	0.055555555555555594
  (5, 3)	-0.027777777777777797
  (5, 4)	0.055555555555555594
  (5, 5)	0.05555555555555566
[[ 0.22222222  0.05555556  0.          0.          0.          0.        ]
 [ 0.05555556  0.11111111  0.05555556 -0.02777778  0.          0.        ]
 [ 0.          0.05555556  0.22222222  0.05555556  0.          0.        ]
 [ 0.         -0.02777778  0.05555556  0.11111111  0.05555556 -0.02777778]
 [ 0.          0.          0.          0.05555556  0.22222222  0.05555556]
 [ 0.         

In [15]:
print(f'stiffK(a, c, xh, 2, 2)=')
print(f'{stiffK(a, c, xh, 2, 2)}')
K = stiffK(a, c, xh, 2, 2).toarray()
print(K)

stiffK(a, c, xh, 2, 2)=
  (0, 0)	8.944444444444443
  (0, 1)	-6.749999999999998
  (1, 0)	-6.749999999999998
  (1, 1)	16.5
  (1, 2)	-12.083333333333332
  (1, 3)	1.8680555555555547
  (2, 1)	-12.083333333333332
  (2, 2)	30.388888888888886
  (2, 3)	-18.05555555555555
  (3, 1)	1.8680555555555545
  (3, 2)	-18.05555555555555
  (3, 3)	37.55555555555557
  (3, 4)	-24.722222222222253
  (3, 5)	3.5208333333333535
  (4, 3)	-24.722222222222257
  (4, 4)	57.16666666666675
  (4, 5)	-32.02777777777782
  (5, 3)	3.5208333333333535
  (5, 4)	-32.02777777777782
  (5, 5)	28.631944444444475
[[  8.94444444  -6.75         0.           0.           0.
    0.        ]
 [ -6.75        16.5        -12.08333333   1.86805556   0.
    0.        ]
 [  0.         -12.08333333  30.38888889 -18.05555556   0.
    0.        ]
 [  0.           1.86805556 -18.05555556  37.55555556 -24.72222222
    3.52083333]
 [  0.           0.           0.         -24.72222222  57.16666667
  -32.02777778]
 [  0.           0.           0.      